In [ ]:
import tensorflow as tf
from utils_plot import plot_series, plot_train_history, plot_prediction
print(tf.__version__)

## Load data

In [ ]:
import numpy as np
import pandas as pd

df_impressionist_sorted = pd.read_csv('impressionsist_sorted.csv')
impressionist_sorted_matrix = np.load('impressionist_sorted_matrix.npy')

In [ ]:
time = np.arange(impressionist_sorted_matrix.shape[0])

## Split dataset

**Define window size**

In [ ]:
split_time = 4500

#Define feature
x = impressionist_sorted_matrix[:,0]

#split time serie
time_train = time[:split_time]
time_valid = time[split_time:]


window_size = 20
batch_size = 256
shuffle_buffer_size = 1000

**Create windowed dataset**

In [ ]:
from Prediction_model_feature import Windowed_Dataset

In [ ]:
dataset = Windowed_Dataset(x,
                           split_time=split_time,
                           window_size=window_size, 
                           shuffle_buffer=shuffle_buffer_size,
                           train_batch_size=batch_size,
                            val_batch_size=batch_size)

In [ ]:
train_set = dataset.get_train_dataset()
val_set = dataset.get_val_dataset()

**Plot example**

In [ ]:
for x, y in train_set.take(1):
    plot = plot_prediction([x[0].numpy(), y[0].numpy()] , 'Sample example')
    plot.show()

## Define model

In [ ]:
def define_model():

    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    np.random.seed(51)

    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                          strides=1, padding="causal",
                          activation="relu",
                          input_shape=[None, 1]),
      tf.keras.layers.LSTM(window_size, return_sequences=True),
      tf.keras.layers.LSTM(window_size, return_sequences=True),
      tf.keras.layers.Dense(30, activation="relu"),
      tf.keras.layers.Dense(10, activation="relu"),
      tf.keras.layers.Dense(1),
      tf.keras.layers.Lambda(lambda x: x * 400)
    ],
    name="Sequence_Feature_1")
    
    return model

In [ ]:
model = define_model()
model.summary()

**Train with different learning rates**

In [ ]:
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

In [ ]:
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

**Find best learning rate**

In [ ]:
import matplotlib.pyplot as plt

plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 60])

In [ ]:
min_index = np.argmin(np.array(history.history["loss"]))
lr = history.history['lr'][min_index]

In [ ]:
#Best learning rate. WIth this value, the model overfits
lr

**Train with best learning rate**

In [ ]:
model = define_model()

optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set,
                    epochs=20,
                    validation_data=val_set)

In [ ]:
plot_train_history(history, "Train history")

## Evaluate model

**Predict time series with the model**

In [ ]:
for x, y in val_set.take(3):
    print(x.shape)
    print(y.shape)

**Plot one prediction** 

In [ ]:
for x, y in val_set.take(1):
    prediction = model.predict(x)[0]
    plot = plot_prediction([x[0].numpy(), y[0].numpy(), prediction[0]] , 'Simple LSTM model')
    plot.show()

**Plot all prediction future**

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.map(lambda w: (w[:]))
    ds = ds.batch(batch_size)
    forecast = model.predict(ds)
    return forecast

In [ ]:
x = impressionist_sorted_matrix[:,0]
rnn_forecast = model_forecast(model, x, window_size, batch_size)


In [ ]:
rnn_forecast = rnn_forecast[split_time-window_size+1:,-1,0]

In [ ]:
rnn_forecast.shape

**Plot series**

In [ ]:
#plot_series(time_train, x_train)
plot_series(time_valid, [(rnn_forecast, 'rnn')])


In [ ]:
x_valid = impressionist_sorted_matrix[:,0][split_time:]
plot_series(time_valid, [(x_valid, 'x_valid'), (rnn_forecast, 'rnn')])

In [ ]:
tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy().mean()